In [1]:
import pandas as   pd

In [2]:
data=pd.read_csv("crop_prediction.csv")


In [3]:
data

,N,P,K,temperature,humidity,ph,rainfall,crop
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [22]:
train_data=data.drop("label",axis=1)
test_data=data["label"]
train_data
test_data

0         rice
1         rice
2         rice
3         rice
4         rice
         ...  
2195    coffee
2196    coffee
2197    coffee
2198    coffee
2199    coffee
Name: label, Length: 2200, dtype: object

In [23]:
train_data.to_csv('train.csv', index=False, header=True)
test_data.to_csv('test.csv', index=False, header=True)

In [24]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install sagemaker

Note: you may need to restart the kernel to use updated packages.


In [26]:
import boto3
session = boto3.Session()
session

Session(region_name='us-west-2')

In [27]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_SESSION_TOKEN'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

In [30]:
pip install awscli

     ---------------------------------------- 4.5/4.5 MB 15.2 MB/s eta 0:00:00
     ---------------------------------------- 548.2/548.2 kB ? eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
Note: you may need to restart the kernel to use updated packages.


In [39]:
import sagemaker, boto3, os
bucket ='qs-sm-customer-churn-data'
prefix = "demo-sagemaker-xgboost-crop-predict"
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'data/train.csv')).upload_file('train.csv')


In [40]:
! aws s3 ls {bucket}/{prefix}/data --recursive

In [41]:
import sagemaker

region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

AWS Region: us-west-2
RoleArn: arn:aws:iam::061051234909:role/aws-reserved/sso.amazonaws.com/us-west-2/AWSReservedSSO_Gen-AI-Dev-Team_8a7b9f3f1536f7ba


In [6]:
import os

# Set AWS credentials in environment variables
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_SESSION_TOKEN'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
df = pd.read_csv('crop_prediction.csv')
df.head()

,N,P,K,temperature,humidity,ph,rainfall,crop
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [49]:
label_encoder = LabelEncoder()
df['crop'] = label_encoder.fit_transform(df['crop'])
df

,N,P,K,temperature,humidity,ph,rainfall,crop
0,90,42,43,20.879744,82.002744,6.502985,202.935536,20
1,85,58,41,21.770462,80.319644,7.038096,226.655537,20
2,60,55,44,23.004459,82.320763,7.840207,263.964248,20
3,74,35,40,26.491096,80.158363,6.980401,242.864034,20
4,78,42,42,20.130175,81.604873,7.628473,262.717340,20
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,5
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,5
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,5
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,5


In [51]:
X = df.drop('crop', axis=1)  # Features
y = df['crop']  # Target

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
#train_file = 'train.csv'
#test_file = 'test.csv'

#X_train.to_csv(train_file, index=False)
#X_test.to_csv(test_file, index=False)

import sagemaker, boto3, os
bucket = "sagemaker-us-west-2-061051234909"
prefix = "demo-sagemaker-xgboost-adult-income-predict"

boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/test.csv')).upload_file('test.csv')

INFO:botocore.credentials:Found credentials in environment variables.
INFO:botocore.credentials:Found credentials in environment variables.


In [80]:
from sagemaker.inputs import TrainingInput
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
import sagemaker

# Define the S3 path for the training data
s3_train_path = f's3://sagemaker-us-west-2-061051234909/demo-sagemaker-xgboost-adult-income-predict\data/train.csv'
s3_output_location='s3://{}/{}/{}'.format(bucket, prefix, 'xgboost_model')

# XGBoost container
container = sagemaker.image_uris.retrieve("xgboost", region='us-west-2', version="1.2-1")
print(container)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1


In [81]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri=container,
    role='arn:aws:iam::061051234909:role/service-role/AmazonSageMaker-ExecutionRole-20241010T155033',
    #role=role,
    instance_count=1,
    #instance_type='ml.m5.xlarge',  # You can modify the instance type as needed
    instance_type='ml.p3.2xlarge',
    volume_size=5,
    max_run=3600,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session(),
    rules=[
        Rule.sagemaker(rule_configs.create_xgboost_report()),
        ProfilerRule.sagemaker(rule_configs.ProfilerReport())
    ]
)


In [82]:
xgb_model.set_hyperparameters(
    objective='multi:softmax',  # Assuming you're predicting categories (crop types)
    num_class=len(label_encoder.classes_),  # Number of unique crops
    num_round=100,  # You can adjust this based on your dataset
    max_depth=5,           # Maximum depth of a tree
    eta=0.2,               # Step size shrinkage
    subsample=0.8,         # Subsample ratio of the training instances
    colsample_bytree=0.8,  # Subsample ratio of columns
    eval_metric='auc',     # Evaluation metric
    tree_method='gpu_hist' 
)


In [83]:
train_input = TrainingInput(s3_train_path, content_type='csv')

In [84]:
xgb_model.fit({'train': train_input})

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-10-19-20-32-05-287
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://sagemaker-us-west-2-061051234909/demo-sagemaker-xgboost-adult-income-predict/data/train.csv" given in input data source. Please ensure that the bucket exists in the selected region (us-west-2), that objects exist under that S3 prefix, and that the role "arn:aws:iam::061051234909:role/service-role/AmazonSageMaker-ExecutionRole-20241010T155033" has "s3:ListBucket" permissions on bucket "sagemaker-us-west-2-061051234909".

In [2]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker import image_uris, get_execution_role
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from io import StringIO
import time

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\SANDEEP\AppData\Local\sagemaker\sagemaker\config.yaml


In [12]:
role

NameError: name 'role' is not defined

In [13]:
df=pd.read_csv('crop_prediction.csv')
df

,N,P,K,temperature,humidity,ph,rainfall,crop
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [14]:
X = df.drop('crop', axis=1)  # Replace 'target' with your target column name
y = df['crop']


In [44]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels to convert them to numerical format
y = label_encoder.fit_transform(y)
y=pd.Series(y)
y.name='crop'
print(y)

0       20
1       20
2       20
3       20
4       20
        ..
2195     5
2196     5
2197     5
2198     5
2199     5
Name: crop, Length: 2200, dtype: int64


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (1760, 7)
Testing set shape: (440, 7)


In [65]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [68]:

train_data = pd.concat([y_train, X_train], axis=1)
test_data = pd.concat([y_test, X_test], axis=1)
val_data=pd.concat([y_val,X_val],axis=1)

print(train_data)

      crop    N   P   K  temperature   humidity        ph    rainfall
583     13   29  41  21    31.493981  62.849169  8.869797   64.568076
799      2   38  62  25    32.747739  67.779546  7.453975   63.377844
1125    12   12  37  30    31.097791  47.411967  4.546466   90.286243
34      20   88  54  44    25.735429  83.882662  6.149411  233.132137
1426    15   85  21  52    29.628007  90.100516  6.075144   23.695868
...    ...  ...  ..  ..          ...        ...       ...         ...
1065     1  104  80  54    27.090622  81.335069  5.879119  110.133118
1624    16   33  14   8    21.032001  92.964197  7.684420  110.682394
1979     6  126  46  25    24.438474  81.698017  6.757458   60.796459
437     18   17  64  16    30.977587  32.249142  7.161798  180.716828
1690    16   20  29  10    29.074127  93.271891  7.365492  100.789687

[1408 rows x 8 columns]


In [69]:
train_data.to_csv('train_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)
val_data.to_csv('val_data.csv',index=False,header=False)

In [70]:
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'crop-prediction-linear'
bucket

'sagemaker-us-west-2-061051234909'

In [59]:
import sagemaker

region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))


AWS Region: us-west-2
RoleArn: arn:aws:iam::061051234909:role/aws-reserved/sso.amazonaws.com/us-west-2/AWSReservedSSO_Gen-AI-Dev-Team_8a7b9f3f1536f7ba


In [73]:
s3 = boto3.client('s3')
train_file_key = f'{prefix}/train/train_data.csv'
test_file_key = f'{prefix}/test/test_data.csv'
val_file_key=f'{prefix}/val/val_data.csv'

In [74]:
s3.upload_file('train_data.csv', bucket, train_file_key)
s3.upload_file('test_data.csv', bucket, test_file_key)
s3.upload_file('val_data.csv', bucket, val_file_key)

In [75]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.session import TrainingInput

s3_output_location='s3://{}/{}/{}'.format(bucket, prefix, 'xgboost_model')

container=sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
print(container)

xgb_model=sagemaker.estimator.Estimator(
    image_uri=container,
    role='arn:aws:iam::061051234909:role/service-role/AmazonSageMaker-ExecutionRole-20241010T155033',
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session(),
    rules=[
        Rule.sagemaker(rule_configs.create_xgboost_report()),
        ProfilerRule.sagemaker(rule_configs.ProfilerReport())
    ]
)

246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1


In [84]:
xgb_model.set_hyperparameters(
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    objective = "multi:softmax",
    num_round = 1000,
    num_class=22,
    #tree_method='gpu_hist'
)

In [85]:
from sagemaker.session import TrainingInput

train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train/train_data.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "val/val_data.csv"), content_type="csv"
)
print(train_input)

In [87]:
xgb_model.fit({"train": train_input, "validation": validation_input}, wait=True)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-10-23-03-10-43-940


2024-10-23 03:10:44 Starting - Starting the training job
2024-10-23 03:10:44 Pending - Training job waiting for capacity...CreateXgboostReport: InProgress
ProfilerReport: InProgress
...
2024-10-23 03:11:42 Pending - Preparing the instances for training...
2024-10-23 03:12:17 Downloading - Downloading the training image......
2024-10-23 03:13:23 Training - Training image download completed. Training in progress....
2024-10-23 03:13:52 Uploading - Uploading generated training model
2024-10-23 03:13:52 Failed - Training job failed
[2024-10-23 03:13:39.821 ip-10-0-208-213.us-west-2.compute.internal:13 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter tree_method value gpu_hist to Json.
Returning the value itself
INFO:sagemaker_x

UnexpectedStatusException: Error for Training job sagemaker-xgboost-2024-10-23-03-10-43-940: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_algorithm_toolkit/hyperparameter_validation.py", line 278, in validate
    hyperparameter_obj = self.hyperparameters[hp]
KeyError: 'num_classes'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_xgboost_container/training.py", line 94, in main
    train(framework.training_env())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_xgboost_container/training.py", line 90, in train
    run_algorithm_mode()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_xgboost_container/training.py", line 68, in run_algorithm_mode

In [145]:
multiclass_estimator.set_hyperparameters(
    mini_batch_size=16,
    epochs=10,
    feature_dim=X_train.shape[1],  # Number of feature  # Number of classes
    optimizer='adam',
             # Label normalization disabled
)

AttributeError: 'LinearLearner' object has no attribute 'set_hyperparameters'

In [48]:
train_s3 = upload_to_s3(X_train, y_train, 'train')
test_s3 = upload_to_s3(X_test, y_test, 'validation')

In [49]:

print("Training data uploaded to:", train_s3)
print("Validation data uploaded to:", test_s3)


Training data uploaded to: s3://sagemaker-us-west-2-061051234909/crop-prediction-linear/train/train.csv
Validation data uploaded to: s3://sagemaker-us-west-2-061051234909/crop-prediction-linear/validation/validation.csv


In [50]:
role = get_execution_role()

In [51]:
container = image_uris.retrieve(
    framework='linear-learner',
    region=session.boto_region_name
)

In [52]:
num_classes = len(y.columns)

In [57]:
estimator = sagemaker.estimator.Estimator(
    container,
    role='arn:aws:iam::061051234909:role/service-role/AmazonSageMaker-ExecutionRole-20241010T155033',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=f's3://{bucket}/{prefix}/output',
    sagemaker_session=session
)

In [146]:
multiclass_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='multiclass_classifier',
                                               num_classes=7)

ClientError: An error occurred (ExpiredToken) when calling the GetCallerIdentity operation: The security token included in the request is expired

In [60]:

estimator.set_hyperparameters(
    feature_dim=X_train.shape[1],  # Number of features
    predictor_type='multiclass_classifier',
    num_classes=num_classes,
    mini_batch_size=16,  # Reduced batch size
    epochs=10,
    learning_rate=0.05,
    optimizer='adam',
    normalize_data=False,  # We already normalized the data
    normalize_label=False
)

In [61]:
estimator.fit({
    'train': train_s3,
    'validation': test_s3
})


INFO:sagemaker:Creating training-job with name: linear-learner-2024-10-22-22-15-39-778


2024-10-22 22:15:43 Starting - Starting the training job...
2024-10-22 22:15:56 Starting - Preparing the instances for training...
2024-10-22 22:16:41 Downloading - Downloading input data...
2024-10-22 22:17:11 Downloading - Downloading the training image......
2024-10-22 22:18:17 Training - Training image download completed. Training in progress.....Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/22/2024 22:18:45 INFO 140649955366720] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss'

UnexpectedStatusException: Error for Training job linear-learner-2024-10-22-22-15-39-778: Failed. Reason: ClientError: Unable to read data channel 'train'. Requested content-type is 'application/x-recordio-protobuf'. Please verify the data matches the requested content-type. (caused by MXNetError)

Caused by: [22:18:48] /opt/brazil-pkg-cache/packages/AIAlgorithmsCppLibs/AIAlgorithmsCppLibs-2.0.7138.0/AL2_x86_64/generic-flavor/src/src/aialgs/io/iterator_base.cpp:100: (Input Error) The header of the MXNet RecordIO record at position 0 in the dataset does not start with a valid magic number.

Stack trace returned 10 entries:
[bt] (0) /opt/amazon/lib/libaialgs.so(+0x9d1b) [0x7feb9cef2d1b]
[bt] (1) /opt/amazon/lib/libaialgs.so(+0xa549) [0x7feb9cef3549]
[bt] (2) /opt/amazon/lib/libaialgs.so(aialgs::iterator_base::Next()+0x448) [0x7feb9cf00128]
[bt] (3) /opt/amazon/lib/libmxnet.so(MXDataIterNext+0x21) [0x7feb83559901]
[bt] (4) /opt/amazon/lib/libffi.so.6(ffi_call_unix64+0x4c) [0x7feb9cfe2078]